In [ ]:
import keras
t=keras.models.load_model("weights.best.hdf5")  #loading the saved model for analysis

In [ ]:
import tensorflow as tf
import keras.backend as K
out_layer=t.layers[14].output
output_fn= K.function([t.input],out_layer)
o=output_fn([M_train,x_train])
o.shape                                       # to get output from intermediate layers

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(o)
o=scaler.transform(o)
print(o.shape)

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=3)
pca.fit(o)
x_pca=pca.transform(o)
print(x_pca.shape)
X=x_pca[:,0]
Y=x_pca[:,1]
Z=x_pca[:,1]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig= plt.figure(figsize=(11,11))
ax=plt.axes(projection='3d')   #3D plot of PCA coordinates tranformed from the output of intermediate layers
ax.scatter(X,Y,Z,c=y_train,marker='^')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
plt.show()

In [ ]:
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
#2D plot of PCA coordinates tranformed from the output of intermediate layers
fig, (ax1) = plt.subplots(1,1,figsize=(8,8))
ax1 = plt.axes()
ax1.scatter(X[y_train==1],Y[y_train==1], marker='<',color='blue',s=100)
ax1.scatter(X[y_train==2],Y[y_train==2], marker='s',color='orange',s=100)
ax1.scatter(X[y_train==3],Y[y_train==3], marker='o',color='green',s=100)
ax1.scatter(X[y_train==0],Y[y_train==0], marker='D',color='red',s=100)
ax1.set_xlabel('x axis',fontsize=20)
ax1.set_ylabel('y axis',fontsize=20)

a = mlines.Line2D([], [], color='red', marker='D', linestyle='None',
                          markersize=10, label='Angry')
b = mlines.Line2D([], [], color='blue', marker='<', linestyle='None',
                          markersize=10, label='Happy')
c = mlines.Line2D([], [], color='orange', marker='s', linestyle='None',
                          markersize=10, label='Sad')
d = mlines.Line2D([], [], color='green', marker='o', linestyle='None',
                          markersize=10, label='Neutral')

plt.legend(handles=[a,b,c,d],fontsize=14)
plt.rcParams["axes.labelsize"] = 22
fig.text(0.5, -0.005, "Output Layer", ha='center',fontsize=18)
plt.savefig('PCALastttt(size100).pdf',bbox_inches ="tight", 
            pad_inches = 1, 
            orientation ='landscape',dpi=1200)
plt.show()

In [ ]:
########### CODE RELATED TO INTERSECT MATRIX #############
##########################################################

angry,happy,sad,neutral=[],[],[],[]
for i in range (x_pca.shape[0]):
    if(y_train[i]==3):
        neutral.append(x_pca[i])
    if(y_train[i]==0):
        angry.append(x_pca[i])
    if(y_train[i]==1):
        happy.append(x_pca[i])  
    if(y_train[i]==2):
        sad.append(x_pca[i])    
angry=np.array(angry) 
sad=np.array(sad) 
happy=np.array(happy) 
neutral=np.array(neutral)

In [ ]:
angstd=np.std(angry,axis=0)         #calculating standard devation of the spread for each class
sadstd=np.std(sad,axis=0)
happystd=np.std(happy,axis=0)
neutralstd=np.std(neutral,axis=0)

In [ ]:
angm=np.mean(angry,axis=0)         #calculating mean of the spread for each class
sadm=np.mean(sad,axis=0)
happym=np.mean(happy,axis=0)
neutralm=np.mean(neutral,axis=0)

In [ ]:
angl=angm-2*angstd                 #approximating the left extreme of the spread for each class
sadl=sadm-2*sadstd
happyl=happym-2*happystd
neutrall=neutralm-2*neutralstd

In [ ]:
angr=angm+2*angstd                 #approximating the right extreme of the spread for each class
sadr=sadm+2*sadstd
happyr=happym+2*happystd
neutralr=neutralm+2*neutralstd

In [ ]:
left=np.zeros((4,3))              #storing values of extremes for each class in left and right matrices
left[0]=angl
left[1]=happyl
left[2]=sadl
left[3]=neutrall
right=np.zeros((4,3))
right[0]=angr
right[1]=happyr
right[2]=sadr
right[3]=neutralr

In [ ]:
Xx=np.zeros((4,4))
Yy=np.zeros((4,4))
Zz=np.zeros((4,4))
      
                                #calculating matrices for each of the coordinates
for i in range(0,4):
    for j in range(0,4):
        Xx[i][j]= max((min(right[j][0],right[i][0])-
                       max(left[j][0],left[i][0])),0)/(max(right[j][0],right[i][0])-min(left[j][0],left[i][0]))

for i in range(0,4):
    for j in range(0,4):
        Yy[i][j]= max((min(right[j][1],right[i][1])-
                       max(left[j][1],left[i][1])),0)/(max(right[j][1],right[i][1])-min(left[j][1],left[i][1]))

for i in range(0,4):
    for j in range(0,4):
        Zz[i][j]= max((min(right[j][2],right[i][2])-
                       max(left[j][2],left[i][2])),0)/(max(right[j][2],right[i][2])-min(left[j][2],left[i][2]))

In [ ]:
I=np.zeros((4,4))
I=np.multiply(Xx,np.multiply(Yy,Zz))  #final intersection matrix I "element wise product of Xx,Yy,Zz"

In [ ]:
import seaborn as sb

# plotting intersection matrices using heatmaps

fig, (ax1, ax2,ax3) = plt.subplots(1,3,figsize=(33,8))
sb.set(font_scale=2)
x_axis_labels = ["Angry","Happy","Sad","Neutral"] # labels for x-axis
y_axis_labels = ["Angry","Happy","Sad","Neutral"]
#akws = {"ha": 'left',"va": 'top'}
cbar_kws = {"orientation":"vertical", 
            "ticks":[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1], 
           }
#plt.subplot(1,3,1)
sb.heatmap(Ithird,annot=True,cmap="YlGnBu",fmt='.3f',cbar_kws=cbar_kws,xticklabels=x_axis_labels,linewidths=1, yticklabels=y_axis_labels,ax=ax1)
#plt.subplot(1,3,2)
sb.heatmap(Isecond,annot=True,cmap="YlGnBu",fmt='.3f',cbar_kws=cbar_kws,xticklabels=x_axis_labels,linewidths=1,yticklabels=y_axis_labels,ax=ax2)
#plt.subplot(1,3,3)
sb.heatmap(Ifirst,annot=True,cmap="YlGnBu",fmt='.3f',cbar_kws=cbar_kws,xticklabels=x_axis_labels,linewidths=1, yticklabels=y_axis_labels,ax=ax3)
#h3.set_yticklabels(h3.get_yticklabels(), rotation=0)
ax1.tick_params(rotation=0)
ax1.text(1.3, 4.8, "Third Last Layer",fontsize=22)
ax2.text(1.3, 4.8, "Second Last Layer",fontsize=22)
ax3.text(1.5, 4.8, "Output Layer",fontsize=22)
ax2.tick_params(labelrotation=0)
ax3.tick_params(labelrotation=0)
plt.show()

In [ ]:
###tensorboard visualization###

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle # if your feature vector is stored in pickle file
import tensorflow_datasets as tfds
from tensorboard.plugins import projector
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
emotion = ["Neutral","Angry","Happy","Sad"]

PATH = os.getcwd()

LOG_DIR = PATH + '/log-1/'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')

Voices = tf.Variable(o.reshape((len(o), -1)), name='Voices')

#def save_metadata(file):
with open(metadata, 'w') as metadata_file:
    for row in range(912):
        c = emotion[y_train[row]]
        metadata_file.write('{}\n'.format(c))

        
with tf.compat.v1.Session() as sess:
    saver = tf.compat.v1.train.Saver([Voices])

    sess.run(Voices.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'model.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = Voices.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.compat.v1.summary.FileWriter(LOG_DIR), config)

In [ ]:
!tensorboard --logdir=./log-1/